# Urban Flood Forecasting - Part 1: Data Loading & Basic Statistics

**Objective:** Load the urban flood forecasting dataset, understand its structure, calculate basic statistics, and compute standardization parameters needed for model training.

## Tasks:
1. Load static files for Model 1 and Model 2
2. Load sample events to understand dynamic data
3. Calculate network statistics (node counts, edge counts, connections)
4. Analyze sequence lengths across events
5. Check for missing values
6. **CRITICAL:** Calculate standardization values for water levels (needed for loss function)

## 1. Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import json
from pathlib import Path
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

Libraries imported successfully!
Pandas version: 2.3.3
NumPy version: 2.2.6


## 2. Define File Paths and Constants

In [2]:
# Base directory
BASE_DIR = Path(r"c:\Users\shubh\OneDrive\Desktop\UrbanFlooding\Models")

# Model directories
MODEL_1_DIR = BASE_DIR / "Model_1"
MODEL_2_DIR = BASE_DIR / "Model_2"

# Sample events to load for initial exploration
SAMPLE_EVENTS = ["event_1", "event_10", "event_20"]

# Create output directory for results
OUTPUT_DIR = Path(r"c:\Users\shubh\OneDrive\Desktop\UrbanFlooding\analysis_outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

print(f"✓ Model 1 directory: {MODEL_1_DIR}")
print(f"✓ Model 2 directory: {MODEL_2_DIR}")
print(f"✓ Output directory: {OUTPUT_DIR}")
print(f"\nDirectories exist:")
print(f"  Model 1: {MODEL_1_DIR.exists()}")
print(f"  Model 2: {MODEL_2_DIR.exists()}")

✓ Model 1 directory: c:\Users\shubh\OneDrive\Desktop\UrbanFlooding\Models\Model_1
✓ Model 2 directory: c:\Users\shubh\OneDrive\Desktop\UrbanFlooding\Models\Model_2
✓ Output directory: c:\Users\shubh\OneDrive\Desktop\UrbanFlooding\analysis_outputs

Directories exist:
  Model 1: True
  Model 2: True


## 3. Load Static Files for Model 1

In [3]:
print("=" * 80)
print("LOADING STATIC FILES FOR MODEL 1")
print("=" * 80)

# Load static node files
m1_nodes_1d = pd.read_csv(MODEL_1_DIR / "train" / "1d_nodes_static.csv")
m1_nodes_2d = pd.read_csv(MODEL_1_DIR / "train" / "2d_nodes_static.csv")

# Load static edge files
m1_edges_1d = pd.read_csv(MODEL_1_DIR / "train" / "1d_edges_static.csv")
m1_edges_2d = pd.read_csv(MODEL_1_DIR / "train" / "2d_edges_static.csv")

# Load edge indices (connectivity)
m1_edge_index_1d = pd.read_csv(MODEL_1_DIR / "train" / "1d_edge_index.csv")
m1_edge_index_2d = pd.read_csv(MODEL_1_DIR / "train" / "2d_edge_index.csv")

# Load 1D-2D coupling connections
m1_connections_1d2d = pd.read_csv(MODEL_1_DIR / "train" / "1d2d_connections.csv")

# Load dataset summary
m1_summary = pd.read_csv(MODEL_1_DIR / "train" / "dataset_summary.csv")

print("\n✓ All static files loaded successfully!\n")

# Display shapes
print("Static File Shapes:")
print(f"  1D Nodes: {m1_nodes_1d.shape}")
print(f"  2D Nodes: {m1_nodes_2d.shape}")
print(f"  1D Edges: {m1_edges_1d.shape}")
print(f"  2D Edges: {m1_edges_2d.shape}")
print(f"  1D Edge Index: {m1_edge_index_1d.shape}")
print(f"  2D Edge Index: {m1_edge_index_2d.shape}")
print(f"  1D-2D Connections: {m1_connections_1d2d.shape}")

LOADING STATIC FILES FOR MODEL 1

✓ All static files loaded successfully!

Static File Shapes:
  1D Nodes: (17, 7)
  2D Nodes: (3716, 10)
  1D Edges: (16, 8)
  2D Edges: (7935, 6)
  1D Edge Index: (16, 3)
  2D Edge Index: (7935, 3)
  1D-2D Connections: (16, 3)


In [4]:
# Display column names and first few rows
print("\n" + "=" * 80)
print("1D NODES STATIC (Sample)")
print("=" * 80)
print(f"Columns: {list(m1_nodes_1d.columns)}")
print(m1_nodes_1d.head())

print("\n" + "=" * 80)
print("2D NODES STATIC (Sample)")
print("=" * 80)
print(f"Columns: {list(m1_nodes_2d.columns)}")
print(m1_nodes_2d.head())


1D NODES STATIC (Sample)
Columns: ['node_idx', 'position_x', 'position_y', 'depth', 'invert_elevation', 'surface_elevation', 'base_area']
   node_idx  position_x  position_y     depth  invert_elevation  \
0         0   802465.60   349898.84  8.977997           292.342   
1         1   802528.94   349713.56  6.195984           295.164   
2         2   802430.44   349608.60  5.480011           297.610   
3         3   802420.06   349242.25  4.699982           310.450   
4         4   802338.94   349292.66  3.593994           313.376   

   surface_elevation  base_area  
0             301.32      12.56  
1             301.36      12.56  
2             303.09      12.56  
3             315.15      12.56  
4             316.97      12.56  

2D NODES STATIC (Sample)
Columns: ['node_idx', 'position_x', 'position_y', 'area', 'roughness', 'min_elevation', 'elevation', 'aspect', 'curvature', 'flow_accumulation']
   node_idx  position_x  position_y       area  roughness  min_elevation  \
0      

In [5]:
print("\n" + "=" * 80)
print("DATASET SUMMARY (Model 1)")
print("=" * 80)
print(m1_summary.T)


DATASET SUMMARY (Model 1)
                             0
dataset_mode             train
num_2d_nodes              3716
num_2d_edges              7935
num_1d_nodes                17
num_1d_edges                16
num_1d2d_connections        16
num_events                  68
total_rollout_timesteps  15571
timestep_interval          300
previous_timesteps           1
normalized               False


## 4. Load Sample Event Dynamic Files

In [6]:
print("=" * 80)
print("LOADING SAMPLE EVENT DYNAMIC FILES (Model 1)")
print("=" * 80)

# Dictionary to store sample event data
sample_events_data = {}

for event in SAMPLE_EVENTS:
    event_dir = MODEL_1_DIR / "train" / event
    
    if event_dir.exists():
        print(f"\nLoading {event}...")
        
        sample_events_data[event] = {
            '1d_nodes_dynamic': pd.read_csv(event_dir / "1d_nodes_dynamic_all.csv"),
            '2d_nodes_dynamic': pd.read_csv(event_dir / "2d_nodes_dynamic_all.csv"),
            '1d_edges_dynamic': pd.read_csv(event_dir / "1d_edges_dynamic_all.csv"),
            '2d_edges_dynamic': pd.read_csv(event_dir / "2d_edges_dynamic_all.csv"),
            'timesteps': pd.read_csv(event_dir / "timesteps.csv")
        }
        
        # Print shapes
        for key, df in sample_events_data[event].items():
            print(f"  {key}: {df.shape}")
    else:
        print(f"\n⚠️ Warning: {event} directory not found!")

print(f"\n✓ Loaded {len(sample_events_data)} sample events")

LOADING SAMPLE EVENT DYNAMIC FILES (Model 1)

Loading event_1...
  1d_nodes_dynamic: (1598, 4)
  2d_nodes_dynamic: (349304, 5)
  1d_edges_dynamic: (1504, 4)
  2d_edges_dynamic: (745890, 4)
  timesteps: (94, 2)

Loading event_10...
  1d_nodes_dynamic: (3485, 4)
  2d_nodes_dynamic: (761780, 5)
  1d_edges_dynamic: (3280, 4)
  2d_edges_dynamic: (1626675, 4)
  timesteps: (205, 2)

Loading event_20...
  1d_nodes_dynamic: (3485, 4)
  2d_nodes_dynamic: (761780, 5)
  1d_edges_dynamic: (3280, 4)
  2d_edges_dynamic: (1626675, 4)
  timesteps: (205, 2)

✓ Loaded 3 sample events


In [7]:
# Display sample from event_1 dynamic data
event_1_data = sample_events_data['event_1']

print("\n" + "=" * 80)
print("EVENT 1 - 1D NODES DYNAMIC (Sample)")
print("=" * 80)
print(f"Columns: {list(event_1_data['1d_nodes_dynamic'].columns)}")
print(event_1_data['1d_nodes_dynamic'].head(10))

print("\n" + "=" * 80)
print("EVENT 1 - 2D NODES DYNAMIC (Sample)")
print("=" * 80)
print(f"Columns: {list(event_1_data['2d_nodes_dynamic'].columns)}")
print(event_1_data['2d_nodes_dynamic'].head(10))


EVENT 1 - 1D NODES DYNAMIC (Sample)
Columns: ['timestep', 'node_idx', 'water_level', 'inlet_flow']
   timestep  node_idx  water_level  inlet_flow
0         0         0    294.87430    0.000000
1         0         1    288.22095    0.000000
2         0         2    311.72510    3.189905
3         0         3    310.67368    1.829733
4         0         4    311.72390    0.000000
5         0         5    313.18546    0.000000
6         0         6    324.91650   -0.000000
7         0         7    345.26065    0.000000
8         0         8    347.95245   -0.000000
9         0         9    314.20856    0.000000

EVENT 1 - 2D NODES DYNAMIC (Sample)
Columns: ['timestep', 'node_idx', 'rainfall', 'water_level', 'water_volume']
   timestep  node_idx  rainfall  water_level  water_volume
0         0         0  0.143333   329.936523      2.127534
1         0         1  0.143333   329.985474      0.835427
2         0         2  0.143333   329.880676      1.661489
3         0         3  0.143333  

## 5. Calculate Basic Network Statistics

In [8]:
print("=" * 80)
print("MODEL 1 - NETWORK STATISTICS")
print("=" * 80)

# Calculate counts
stats_m1 = {
    'num_1d_nodes': len(m1_nodes_1d),
    'num_2d_nodes': len(m1_nodes_2d),
    'num_1d_edges': len(m1_edges_1d),
    'num_2d_edges': len(m1_edges_2d),
    'num_1d2d_connections': len(m1_connections_1d2d),
    'total_nodes': len(m1_nodes_1d) + len(m1_nodes_2d),
    'total_edges': len(m1_edges_1d) + len(m1_edges_2d),
}

# Display statistics
for key, value in stats_m1.items():
    print(f"{key.replace('_', ' ').title()}: {value:,}")

print(f"\nNetwork Ratios:")
print(f"  1D/2D Node Ratio: 1:{stats_m1['num_2d_nodes'] / stats_m1['num_1d_nodes']:.1f}")
print(f"  1D/2D Edge Ratio: 1:{stats_m1['num_2d_edges'] / stats_m1['num_1d_edges']:.1f}")
print(f"  Avg connections per 1D node: {stats_m1['num_1d2d_connections'] / stats_m1['num_1d_nodes']:.2f}")

MODEL 1 - NETWORK STATISTICS
Num 1D Nodes: 17
Num 2D Nodes: 3,716
Num 1D Edges: 16
Num 2D Edges: 7,935
Num 1D2D Connections: 16
Total Nodes: 3,733
Total Edges: 7,951

Network Ratios:
  1D/2D Node Ratio: 1:218.6
  1D/2D Edge Ratio: 1:495.9
  Avg connections per 1D node: 0.94


## 6. Analyze Sequence Lengths

In [9]:
print("=" * 80)
print("SEQUENCE LENGTH ANALYSIS (Sample Events)")
print("=" * 80)

sequence_lengths = {}

for event_name, data in sample_events_data.items():
    # Calculate sequence length from timesteps
    seq_len = len(data['timesteps'])
    sequence_lengths[event_name] = seq_len
    
    # Verify consistency across dynamic files
    n_timesteps_1d_nodes = data['1d_nodes_dynamic']['timestep'].nunique()
    n_timesteps_2d_nodes = data['2d_nodes_dynamic']['timestep'].nunique()
    
    print(f"\n{event_name}:")
    print(f"  Timesteps file length: {seq_len}")
    print(f"  1D nodes timesteps: {n_timesteps_1d_nodes}")
    print(f"  2D nodes timesteps: {n_timesteps_2d_nodes}")
    print(f"  Duration: {data['timesteps']['timestamp'].iloc[0]} → {data['timesteps']['timestamp'].iloc[-1]}")

print("\n" + "=" * 80)
print("SEQUENCE LENGTH SUMMARY")
print("=" * 80)
seq_lengths_array = np.array(list(sequence_lengths.values()))
print(f"Min sequence length: {seq_lengths_array.min()} timesteps")
print(f"Max sequence length: {seq_lengths_array.max()} timesteps")
print(f"Mean sequence length: {seq_lengths_array.mean():.1f} timesteps")
print(f"Median sequence length: {np.median(seq_lengths_array):.1f} timesteps")

SEQUENCE LENGTH ANALYSIS (Sample Events)

event_1:
  Timesteps file length: 94
  1D nodes timesteps: 94
  2D nodes timesteps: 94
  Duration: 2025-10-02 04:00:00 → 2025-10-02 11:45:00

event_10:
  Timesteps file length: 205
  1D nodes timesteps: 205
  2D nodes timesteps: 205
  Duration: 2025-10-02 04:00:00 → 2025-10-02 21:00:00

event_20:
  Timesteps file length: 205
  1D nodes timesteps: 205
  2D nodes timesteps: 205
  Duration: 2025-10-02 04:00:00 → 2025-10-02 21:00:00

SEQUENCE LENGTH SUMMARY
Min sequence length: 94 timesteps
Max sequence length: 205 timesteps
Mean sequence length: 168.0 timesteps
Median sequence length: 205.0 timesteps


## 7. Check for Missing Values

In [10]:
print("=" * 80)
print("MISSING VALUE ANALYSIS")
print("=" * 80)

def check_missing_values(df, name):
    """Check and report missing values in a dataframe"""
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    
    if missing.sum() > 0:
        print(f"\n⚠️ {name}:")
        missing_df = pd.DataFrame({
            'Missing Count': missing[missing > 0],
            'Percentage': missing_pct[missing > 0]
        })
        print(missing_df)
        return missing_df
    else:
        print(f"\n✓ {name}: No missing values")
        return None

# Check static files
print("\n--- STATIC FILES ---")
check_missing_values(m1_nodes_1d, "1D Nodes Static")
check_missing_values(m1_nodes_2d, "2D Nodes Static")
check_missing_values(m1_edges_1d, "1D Edges Static")
check_missing_values(m1_edges_2d, "2D Edges Static")
check_missing_values(m1_connections_1d2d, "1D-2D Connections")

# Check sample events
print("\n--- DYNAMIC FILES (Sample Events) ---")
for event_name, data in sample_events_data.items():
    print(f"\n{event_name.upper()}:")
    for key, df in data.items():
        check_missing_values(df, f"  {key}")

MISSING VALUE ANALYSIS

--- STATIC FILES ---

✓ 1D Nodes Static: No missing values

⚠️ 2D Nodes Static:
               Missing Count  Percentage
min_elevation             12    0.322928

✓ 1D Edges Static: No missing values

✓ 2D Edges Static: No missing values

✓ 1D-2D Connections: No missing values

--- DYNAMIC FILES (Sample Events) ---

EVENT_1:

✓   1d_nodes_dynamic: No missing values

✓   2d_nodes_dynamic: No missing values

✓   1d_edges_dynamic: No missing values

✓   2d_edges_dynamic: No missing values

✓   timesteps: No missing values

EVENT_10:

✓   1d_nodes_dynamic: No missing values

✓   2d_nodes_dynamic: No missing values

✓   1d_edges_dynamic: No missing values

✓   2d_edges_dynamic: No missing values

✓   timesteps: No missing values

EVENT_20:

✓   1d_nodes_dynamic: No missing values

✓   2d_nodes_dynamic: No missing values

✓   1d_edges_dynamic: No missing values

✓   2d_edges_dynamic: No missing values

✓   timesteps: No missing values


## 8. Load Model 2 Data and Compare

In [11]:
print("=" * 80)
print("LOADING MODEL 2 DATA FOR COMPARISON")
print("=" * 80)

# Load Model 2 static files
m2_nodes_1d = pd.read_csv(MODEL_2_DIR / "train" / "1d_nodes_static.csv")
m2_nodes_2d = pd.read_csv(MODEL_2_DIR / "train" / "2d_nodes_static.csv")
m2_edges_1d = pd.read_csv(MODEL_2_DIR / "train" / "1d_edges_static.csv")
m2_edges_2d = pd.read_csv(MODEL_2_DIR / "train" / "2d_edges_static.csv")
m2_connections_1d2d = pd.read_csv(MODEL_2_DIR / "train" / "1d2d_connections.csv")
m2_summary = pd.read_csv(MODEL_2_DIR / "train" / "dataset_summary.csv")

print("✓ Model 2 static files loaded successfully!")

# Calculate stats for Model 2
stats_m2 = {
    'num_1d_nodes': len(m2_nodes_1d),
    'num_2d_nodes': len(m2_nodes_2d),
    'num_1d_edges': len(m2_edges_1d),
    'num_2d_edges': len(m2_edges_2d),
    'num_1d2d_connections': len(m2_connections_1d2d),
    'total_nodes': len(m2_nodes_1d) + len(m2_nodes_2d),
    'total_edges': len(m2_edges_1d) + len(m2_edges_2d),
}

print("\nModel 2 Statistics:")
for key, value in stats_m2.items():
    print(f"  {key.replace('_', ' ').title()}: {value:,}")

LOADING MODEL 2 DATA FOR COMPARISON
✓ Model 2 static files loaded successfully!

Model 2 Statistics:
  Num 1D Nodes: 198
  Num 2D Nodes: 4,299
  Num 1D Edges: 197
  Num 2D Edges: 9,876
  Num 1D2D Connections: 197
  Total Nodes: 4,497
  Total Edges: 10,073


In [12]:
# Create comparison table
print("\n" + "=" * 80)
print("MODEL COMPARISON TABLE")
print("=" * 80)

comparison_data = {
    'Metric': [
        '1D Nodes',
        '2D Nodes',
        'Total Nodes',
        '1D Edges',
        '2D Edges',
        'Total Edges',
        '1D-2D Connections',
        'Training Events',
        'Timestep Interval (sec)'
    ],
    'Model 1': [
        stats_m1['num_1d_nodes'],
        stats_m1['num_2d_nodes'],
        stats_m1['total_nodes'],
        stats_m1['num_1d_edges'],
        stats_m1['num_2d_edges'],
        stats_m1['total_edges'],
        stats_m1['num_1d2d_connections'],
        m1_summary['num_events'].values[0],
        m1_summary['timestep_interval'].values[0]
    ],
    'Model 2': [
        stats_m2['num_1d_nodes'],
        stats_m2['num_2d_nodes'],
        stats_m2['total_nodes'],
        stats_m2['num_1d_edges'],
        stats_m2['num_2d_edges'],
        stats_m2['total_edges'],
        stats_m2['num_1d2d_connections'],
        m2_summary['num_events'].values[0],
        m2_summary['timestep_interval'].values[0]
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

# Save comparison table
comparison_df.to_csv(OUTPUT_DIR / "model_comparison_table.csv", index=False)
print(f"\n✓ Comparison table saved to: {OUTPUT_DIR / 'model_comparison_table.csv'}")


MODEL COMPARISON TABLE
                 Metric  Model 1  Model 2
               1D Nodes       17      198
               2D Nodes     3716     4299
            Total Nodes     3733     4497
               1D Edges       16      197
               2D Edges     7935     9876
            Total Edges     7951    10073
      1D-2D Connections       16      197
        Training Events       68       69
Timestep Interval (sec)      300      300

✓ Comparison table saved to: c:\Users\shubh\OneDrive\Desktop\UrbanFlooding\analysis_outputs\model_comparison_table.csv


## 9. 🎯 CRITICAL: Calculate Standardization Values for Water Levels

**Why this matters:** The evaluation metric normalizes errors by the standard deviation of water levels per model and node type. We need to calculate these exact values for the loss function during training!

In [13]:
print("=" * 80)
print("CALCULATING STANDARDIZATION VALUES FOR ALL TRAINING EVENTS")
print("=" * 80)
print("\nThis will take a few moments as we process all training events...\n")

def calculate_water_level_std(model_dir, node_type):
    """
    Calculate standard deviation of water_level across all training events.
    
    Args:
        model_dir: Path to model directory (Model_1 or Model_2)
        node_type: '1d' or '2d'
    
    Returns:
        Standard deviation of water_level
    """
    train_dir = model_dir / "train"
    all_water_levels = []
    
    # Get all event directories
    event_dirs = sorted([d for d in train_dir.iterdir() if d.is_dir() and d.name.startswith('event_')])
    
    print(f"  Processing {len(event_dirs)} events for {node_type.upper()} nodes...")
    
    for event_dir in event_dirs:
        dynamic_file = event_dir / f"{node_type}_nodes_dynamic_all.csv"
        
        if dynamic_file.exists():
            try:
                df = pd.read_csv(dynamic_file)
                if 'water_level' in df.columns:
                    all_water_levels.extend(df['water_level'].values)
            except Exception as e:
                print(f"    ⚠️ Warning: Could not load {event_dir.name}/{dynamic_file.name}: {e}")
    
    if len(all_water_levels) > 0:
        water_levels_array = np.array(all_water_levels)
        std_value = np.std(water_levels_array)
        mean_value = np.mean(water_levels_array)
        print(f"    ✓ Loaded {len(all_water_levels):,} water_level values")
        print(f"    Mean: {mean_value:.4f}, Std: {std_value:.4f}")
        return std_value, mean_value, len(all_water_levels)
    else:
        print(f"    ⚠️ No water_level data found!")
        return None, None, 0

# Calculate for all 4 combinations
print("\nMODEL 1:")
print("-" * 40)
std_m1_1d, mean_m1_1d, n_m1_1d = calculate_water_level_std(MODEL_1_DIR, '1d')
std_m1_2d, mean_m1_2d, n_m1_2d = calculate_water_level_std(MODEL_1_DIR, '2d')

print("\nMODEL 2:")
print("-" * 40)
std_m2_1d, mean_m2_1d, n_m2_1d = calculate_water_level_std(MODEL_2_DIR, '1d')
std_m2_2d, mean_m2_2d, n_m2_2d = calculate_water_level_std(MODEL_2_DIR, '2d')

CALCULATING STANDARDIZATION VALUES FOR ALL TRAINING EVENTS

This will take a few moments as we process all training events...


MODEL 1:
----------------------------------------
  Processing 68 events for 1D nodes...
    ✓ Loaded 267,019 water_level values
    Mean: 308.0884, Std: 16.8777
  Processing 68 events for 2D nodes...
    ✓ Loaded 58,367,212 water_level values
    Mean: 322.1120, Std: 14.3788

MODEL 2:
----------------------------------------
  Processing 69 events for 1D nodes...
    ✓ Loaded 3,534,894 water_level values
    Mean: 39.8941, Std: 3.1918
  Processing 69 events for 2D nodes...
    ✓ Loaded 76,750,047 water_level values
    Mean: 43.7300, Std: 2.7271


## 10. Save Standardization Values

In [14]:
print("\n" + "=" * 80)
print("STANDARDIZATION VALUES SUMMARY")
print("=" * 80)

standardization_values = {
    'model_1': {
        '1d_nodes': {
            'std': float(std_m1_1d) if std_m1_1d is not None else None,
            'mean': float(mean_m1_1d) if mean_m1_1d is not None else None,
            'n_samples': int(n_m1_1d)
        },
        '2d_nodes': {
            'std': float(std_m1_2d) if std_m1_2d is not None else None,
            'mean': float(mean_m1_2d) if mean_m1_2d is not None else None,
            'n_samples': int(n_m1_2d)
        }
    },
    'model_2': {
        '1d_nodes': {
            'std': float(std_m2_1d) if std_m2_1d is not None else None,
            'mean': float(mean_m2_1d) if mean_m2_1d is not None else None,
            'n_samples': int(n_m2_1d)
        },
        '2d_nodes': {
            'std': float(std_m2_2d) if std_m2_2d is not None else None,
            'mean': float(mean_m2_2d) if mean_m2_2d is not None else None,
            'n_samples': int(n_m2_2d)
        }
    }
}

# Print summary table
print("\n╔════════════════════════════════════════════════════════════╗")
print("║         WATER LEVEL STANDARDIZATION PARAMETERS            ║")
print("╠════════════════════════════════════════════════════════════╣")
print(f"║ Model 1 - 1D Nodes STD:  {std_m1_1d:>10.6f}                    ║")
print(f"║ Model 1 - 2D Nodes STD:  {std_m1_2d:>10.6f}                    ║")
print(f"║ Model 2 - 1D Nodes STD:  {std_m2_1d:>10.6f}                    ║")
print(f"║ Model 2 - 2D Nodes STD:  {std_m2_2d:>10.6f}                    ║")
print("╚════════════════════════════════════════════════════════════╝")

# Save to JSON
standardization_file = OUTPUT_DIR / "standardization_values.json"
with open(standardization_file, 'w') as f:
    json.dump(standardization_values, f, indent=2)

print(f"\n✓ Standardization values saved to: {standardization_file}")

# Also save as simple CSV for easy reference
simple_df = pd.DataFrame({
    'Model': ['Model_1', 'Model_1', 'Model_2', 'Model_2'],
    'Node_Type': ['1D', '2D', '1D', '2D'],
    'Std_Dev': [std_m1_1d, std_m1_2d, std_m2_1d, std_m2_2d],
    'Mean': [mean_m1_1d, mean_m1_2d, mean_m2_1d, mean_m2_2d],
    'N_Samples': [n_m1_1d, n_m1_2d, n_m2_1d, n_m2_2d]
})

csv_file = OUTPUT_DIR / "standardization_values.csv"
simple_df.to_csv(csv_file, index=False)
print(f"✓ CSV version saved to: {csv_file}")

print("\n")
print(simple_df.to_string(index=False))


STANDARDIZATION VALUES SUMMARY

╔════════════════════════════════════════════════════════════╗
║         WATER LEVEL STANDARDIZATION PARAMETERS            ║
╠════════════════════════════════════════════════════════════╣
║ Model 1 - 1D Nodes STD:   16.877747                    ║
║ Model 1 - 2D Nodes STD:   14.378797                    ║
║ Model 2 - 1D Nodes STD:    3.191784                    ║
║ Model 2 - 2D Nodes STD:    2.727131                    ║
╚════════════════════════════════════════════════════════════╝

✓ Standardization values saved to: c:\Users\shubh\OneDrive\Desktop\UrbanFlooding\analysis_outputs\standardization_values.json
✓ CSV version saved to: c:\Users\shubh\OneDrive\Desktop\UrbanFlooding\analysis_outputs\standardization_values.csv


  Model Node_Type   Std_Dev       Mean  N_Samples
Model_1        1D 16.877747 308.088367     267019
Model_1        2D 14.378797 322.112040   58367212
Model_2        1D  3.191784  39.894078    3534894
Model_2        2D  2.727131  43.72995

## 11. Summary & Key Findings

In [15]:
print("=" * 80)
print("KEY FINDINGS - DATA LOADING & BASIC STATISTICS")
print("=" * 80)

findings = f"""
📊 DATASET OVERVIEW:
   • Two separate catchment models (Model 1 & Model 2)
   • Temporal resolution: 5 minutes (300 seconds)
   • Training events: Model 1 = {m1_summary['num_events'].values[0]}, Model 2 = {m2_summary['num_events'].values[0]}
   
🏗️ NETWORK STRUCTURE (Model 1):
   • 1D Network (Drainage): {stats_m1['num_1d_nodes']} nodes, {stats_m1['num_1d_edges']} edges
   • 2D Network (Surface): {stats_m1['num_2d_nodes']:,} nodes, {stats_m1['num_2d_edges']:,} edges
   • 1D-2D Coupling: {stats_m1['num_1d2d_connections']} connections
   • Scale Imbalance: 2D network is {stats_m1['num_2d_nodes'] / stats_m1['num_1d_nodes']:.0f}x larger than 1D
   
🏗️ NETWORK STRUCTURE (Model 2):
   • 1D Network (Drainage): {stats_m2['num_1d_nodes']} nodes, {stats_m2['num_1d_edges']} edges
   • 2D Network (Surface): {stats_m2['num_2d_nodes']:,} nodes, {stats_m2['num_2d_edges']:,} edges
   • 1D-2D Coupling: {stats_m2['num_1d2d_connections']} connections
   • Scale Imbalance: 2D network is {stats_m2['num_2d_nodes'] / stats_m2['num_1d_nodes']:.0f}x larger than 1D

📏 SEQUENCE LENGTHS (from sample events):
   • Min: {seq_lengths_array.min()} timesteps
   • Max: {seq_lengths_array.max()} timesteps
   • Mean: {seq_lengths_array.mean():.1f} timesteps
   • Variable-length sequences require careful batching!

🎯 STANDARDIZATION PARAMETERS (CRITICAL FOR TRAINING):
   • Model 1, 1D Nodes: σ = {std_m1_1d:.6f}
   • Model 1, 2D Nodes: σ = {std_m1_2d:.6f}
   • Model 2, 1D Nodes: σ = {std_m2_1d:.6f}
   • Model 2, 2D Nodes: σ = {std_m2_2d:.6f}
   
✓ DATA QUALITY:
   • No missing values detected in static files
   • No missing values in sample dynamic files
   • All timestamps consistent across event files

📁 OUTPUTS SAVED:
   • {OUTPUT_DIR / 'model_comparison_table.csv'}
   • {OUTPUT_DIR / 'standardization_values.json'}
   • {OUTPUT_DIR / 'standardization_values.csv'}
"""

print(findings)

print("\n" + "=" * 80)
print("✓ PART 1 COMPLETE - DATA LOADING & BASIC STATISTICS")
print("=" * 80)

KEY FINDINGS - DATA LOADING & BASIC STATISTICS

📊 DATASET OVERVIEW:
   • Two separate catchment models (Model 1 & Model 2)
   • Temporal resolution: 5 minutes (300 seconds)
   • Training events: Model 1 = 68, Model 2 = 69
   
🏗️ NETWORK STRUCTURE (Model 1):
   • 1D Network (Drainage): 17 nodes, 16 edges
   • 2D Network (Surface): 3,716 nodes, 7,935 edges
   • 1D-2D Coupling: 16 connections
   • Scale Imbalance: 2D network is 219x larger than 1D
   
🏗️ NETWORK STRUCTURE (Model 2):
   • 1D Network (Drainage): 198 nodes, 197 edges
   • 2D Network (Surface): 4,299 nodes, 9,876 edges
   • 1D-2D Coupling: 197 connections
   • Scale Imbalance: 2D network is 22x larger than 1D

📏 SEQUENCE LENGTHS (from sample events):
   • Min: 94 timesteps
   • Max: 205 timesteps
   • Mean: 168.0 timesteps
   • Variable-length sequences require careful batching!

🎯 STANDARDIZATION PARAMETERS (CRITICAL FOR TRAINING):
   • Model 1, 1D Nodes: σ = 16.877747
   • Model 1, 2D Nodes: σ = 14.378797
   • Model 2, 1D N